In [46]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [47]:
def pretty_print_docs(docs):
    for i, d in enumerate(docs):
        print(f"문서 {i+1}:\n\n" + d.page_content)
        print(f"{'-' * 100}\n")

In [48]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

In [49]:
from langchain_openai import ChatOpenAI

### **파이프라인 생성(압축기+문서 변환기)**

DocumentCompressorPipeline 을 사용하면 여러 compressor를 순차적으로 결합할 수 있습니다.<br>
절차를 순서로 매겨서 순서대로 실행 되도록 만들어 줄수가 있습니다. <br>

In [50]:
loader = TextLoader("./data/appendix-keywords.txt", encoding='utf-8')

In [51]:
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(splitter)

In [52]:
embeddings = OpenAIEmbeddings(api_key=key)

In [53]:
# 기본 retriever 생성
retriever = FAISS.from_documents(texts, embedding=embeddings).as_retriever()

In [54]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [55]:
# 임베딩을 사용하여 redundant_filter(중복 필터)를 생성합니다.
# 검색된 문서 사이에 유사도(0.95으로 설정됨) 계산을 하고 유사도가 0.95 보다 높은 것들은 중복으로 판단해서 drop
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

In [56]:
# 임베딩을 사용하여 관련성 필터를 생성하고, 유사도 임계값을 0.86으로 설정합니다.
# 유사도 기반으로 0.86 이상인 것들은 걸러준다.
# 관련성 있는 문서만 있도록 필터링한다.
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

In [57]:
# 먼저 문서를 더 작은 청크로 분할한 다음, 중복 문서를 제거하고, 
# 쿼리와의 관련성을 기준으로 필터링하여 compressor pipeline을 생성

In [58]:
pipeline_compressor = DocumentCompressorPipeline(
    # 문서 압축 파이프라인을 생성하고, 분할기, 중복 필터, 관련성 필터, LLMChainExtractor를 변환기로 설정합니다.
    transformers=[
        splitter,               # 분할기 (문서를 조각들로 쪼갠다)
        redundant_filter,       # 중복되는 문서 걸러주는 필터.
        relevant_filter,        # 관련성 있는 문서만 있도록 걸러주는 필터. embeddingsFilter(유사도 기반으로 0.86 이상인 것들은 걸러주는 필터).
        LLMChainExtractor.from_llm(llm),  # 압축
    ]
)

In [59]:
# 기본 압축기로 pipeline_compressor를 사용하고, 
# 기본 검색기로 retriever를 사용하여 ContextualCompressionRetriever를 초기화합니다.
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,    # 기본 압축기로 pipeline_compressor를 사용
    base_retriever=retriever,               # 기본 검색기로 retriever
)

In [60]:
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)

In [61]:
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
----------------------------------------------------------------------------------------------------

